In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from random import randint, uniform
from sklearn.preprocessing import StandardScaler
from deap import base, creator, tools, algorithms

In [9]:
df=pd.read_csv(r'C:\Users\anjan\Downloads\P16-Artificial-Neural-Networks\Part 1 - Artificial Neural Networks\Churn_Modelling.csv')

In [10]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Data Preprocessing

In [12]:
X = df.iloc[:, 3:-1].values
y = df.iloc[:, -1].values

In [13]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [14]:
print(y)

[1 0 1 ... 1 1 0]


#### Encoding categorical data

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [16]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [17]:
### One-hot encoding the geography column

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
     

In [19]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


#### Splitting dataset into training and testing set

In [20]:
from sklearn.model_selection import train_test_split
X, X_test, y_train, y = train_test_split(X, y, test_size = 0.2, random_state = 0)


#### Feature Scaling

In [21]:
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)


#### Define the genetic algorithm settings

In [22]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = 12
        self.hidden_size = 25
        self.output_size = 1
        
    def forward(self,X):
        
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))
    
        def relu(x):
            return np.maximum(0, x)
    
        def softmax(x):
            exps = np.exp(x - np.max(x, axis=1, keepdims=True))
            return exps / np.sum(exps, axis=1, keepdims=True)
    
        def forward_propagation(X, weights_input_hidden, weights_hidden_output):
            hidden_layer_input = np.dot(X, weights_input_hidden)
            hidden_layer_output = relu(hidden_layer_input)
            output_layer_input = np.dot(hidden_layer_output, weights_hidden_output)
            output_layer_output = softmax(output_layer_input)
            return hidden_layer_input, hidden_layer_output, output_layer_input, output_layer_output
    
        hidden_layer_input, hidden_layer_output, output_layer_input, output_layer_output=forward_propagation(X_test_scaled, weights_input_hidden, weights_hidden_output)
    def backward(self, X, y, learning_rate):
        # Perform backward pass to update weights
        m = X.shape[0]  # number of samples
        dZ2 = self.A2 - y  # compute gradient of loss with respect to Z2
        dW2 = (1 / m) * np.dot(self.A1.T, dZ2)  # compute gradient of loss with respect to W2
        db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)  # compute gradient of loss with respect to b2

        dA1 = np.dot(dZ2, self.W2.T)  # compute gradient of loss with respect to A1
        dZ1 = dA1 * self.sigmoid_derivative(self.Z1)  # compute gradient of loss with respect to Z1
        dW1 = (1 / m) * np.dot(X.T, dZ1)  # compute gradient of loss with respect to W1
        db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)  # compute gradient of loss with respect to b1

        # Update weights and biases
        self.W1 -= learning_rate * dW1  # update W1
        self.b1 -= learning_rate * db1  # update b1
        self.W2 -= learning_rate * dW2  # update W2
        self.b2 -= learning_rate * db2  # update b2

    def predict(self, X):
        Z1 = np.dot(X, self.W1) + self.b1  # compute Z1
        A1 = self.sigmoid(Z1)  # apply sigmoid activation to Z1
        Z2 = np.dot(A1, self.W2) + self.b2  # compute Z2
        A2 = self.sigmoid(Z2)  # apply sigmoid activation to Z2
        return A2  # return the predicted probabilities 
    

In [23]:
def selection(population, fitness_scores):
    # Perform selection based on fitness scores
    # Input: population - list of individuals in the population
    #        fitness_scores - list of fitness scores for each individual
    # Output: selected_population - list of selected individuals

    # Compute selection probabilities based on fitness scores
    selection_probs = fitness_scores / np.sum(fitness_scores)
    
    # Use numpy's random.choice to perform weighted random selection
    selected_population_indices = np.random.choice(len(population), size=len(population), p=selection_probs)
    
    # Create a new list to store selected individuals
    selected_population = [population[idx] for idx in selected_population_indices]
    
    return selected_population


def crossover(parent1, parent2):
    # Perform crossover between two parents to create two offspring
    # Input: parent1 - first parent
    #        parent2 - second parent
    # Output: offspring1 - first offspring
    #         offspring2 - second offspring
    
    # Randomly select a crossover point
    crossover_point = np.random.randint(1, len(parent1))
    
    # Perform crossover by exchanging genes between parents
    offspring1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    offspring2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    
    return offspring1, offspring2


def mutation(individual, mutation_prob):
    # Perform mutation on an individual
    # Input: individual - individual to mutate
    #        mutation_prob - probability of mutation
    # Output: mutated_individual - mutated individual
    
    # Randomly decide whether to perform mutation
    if np.random.rand() < mutation_prob:
        # Randomly select a gene to mutate
        mutation_point = np.random.randint(0, len(individual))
        
        # Perform mutation by randomly changing the gene value
        mutated_individual = np.copy(individual)
        mutated_individual[mutation_point] = np.random.uniform(-1, 1)
    else:
        mutated_individual = individual
    
    return mutated_individual


def fitness_evaluation(population, X, y):
    # Evaluate the fitness of each individual in the population
    # Input: population - list of individuals in the population
    #        X - input features for neural network
    #        y - target labels for neural network
    # Output: fitness_scores - list of fitness scores for each individual
    
    fitness_scores = []
    for individual in population:
        # Set the individual's weights in the neural network
        neural_network.set_weights(individual)
        
        # Perform forward pass to get predicted probabilities
        predicted_probs = neural_network.predict(X)
        
        # Compute cross-entropy loss as fitness score
        loss = -np.sum(y * np.log(predicted_probs) + (1 - y) * np.log(1 - predicted_probs))
        fitness_scores.append(loss)
    
    return fitness_scores        

In [24]:
def main():
    # Set hyperparameters for the genetic algorithm
    population_size = 50  # Number of individuals in each generation
    crossover_prob = 0.8  # Probability of crossover operation
    mutation_prob = 0.2  # Probability of mutation operation

    # Create initial population of individuals
    population = []
    
    for i in range(population_size):
        # Initialize weights for the neural network
        input_hidden_weights = np.random.uniform(-1, 1, size=(input_size, hidden_size))
        hidden_output_weights = np.random.uniform(-1, 1, size=(hidden_size, 1))
        
        # Flatten the weights into a single 1D array
        individual = np.concatenate((input_hidden_weights.flatten(), hidden_output_weights.flatten()))
        
        population.append(individual)

    # Iterate through generations
    num_generations=20
    for generation in range(num_generations):
        # Evaluate fitness of individuals in the population
        fitness = fitness_evaluation(population,X,y)

        # Select parents for crossover
        parents = selection(population, fitness)

        # Perform crossover and mutation to create offspring
        offspring = []
        for i in range(population_size // 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]
            if np.random.rand() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                offspring.append(child1)
                offspring.append(child2)
            else:
                offspring.append(parent1)
                offspring.append(parent2)

        # Perform mutation on offspring
        for i in range(population_size):
            if np.random.rand() < mutation_prob:
                offspring[i] = mutation(offspring[i])

        # Replace old population with offspring
        population = offspring

        # Update best solution and display progress
        best_individual = population[np.argmax(fitness)]
        # ... (print/update best solution and display progress)

    # After all generations, retrieve best solution
    best_individual = population[np.argmax(fitness)]
    